## 로컬 환경에서 PDF 검색하기 1단계 step2
- PDF 문서를 로드하고 한국어 임베딩 모델을 적용하여 벡터 DB 생성
- 생성된 벡터 DB 에 검색기를 생성하고 LLM 모델을 추가하여 검색하기 

### - 사용한 임베딩 모델 jhgan/ko-sroberta-multitask
### - 사용한 LLM llama3.2

In [1]:
# 라이브러리 설치 (필요한 경우 실행)
%pip install -U langchain langchain_core sentence-transformers faiss-cpu pymupdf

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

# Step 1: 문서 로드
# loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
loader = PyMuPDFLoader("data/AI기반_인파분석플랫폼구축_제안서.pdf")
docs = loader.load()

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [3]:
# Step 3: SentenceTransformer 모델을 LangChain의 Embeddings 클래스로 감싸기
class KoSentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        """문서 리스트를 벡터로 변환"""
        return self.model.encode(texts, convert_to_numpy=True).tolist()

    def embed_query(self, text):
        """검색 쿼리를 벡터로 변환"""
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]


# Step 4: 모델 로드 및 FAISS 인덱스 생성
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")
faiss_index = FAISS.from_documents(split_documents, embedding_model)

In [4]:
# Step 5: FAISS 검색기 생성 (as_retriever() 사용)
retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [5]:
# 검색기 테스트
query = "포항에서 열리는 축제의 이름이 뭐야?"
retriever.invoke(query)

[Document(id='b004e31f-1884-44ae-bed9-1591360abbed', metadata={'producer': '알PDF', 'creator': '알PDF', 'creationdate': '2024-06-09T21:45:38+09:00', 'source': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'file_path': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'total_pages': 113, 'format': 'PDF 1.7', 'title': '합치기_2', 'author': '유선철', 'subject': '', 'keywords': '', 'moddate': '2024-06-09T21:57:34+09:00', 'trapped': '', 'page': 10}, page_content='방문객수\n연도\n151만명\n2019 년\n189만명\n2018 년\n203만명\n2017 년\n187만명\n2016 년\n103만명\n2015 년\n방문객수\n행사명\n연도\n3.5만명\n청년문화페스티벌\n2023 년\n1.5만명\n붐업페스티벌\n2018 년\n-\n-\n-\n-\n-\n-'),
 Document(id='4b1f4c24-e87c-4815-8245-26684b007bf4', metadata={'producer': '알PDF', 'creator': '알PDF', 'creationdate': '2024-06-09T21:45:38+09:00', 'source': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'file_path': 'data/AI기반_인파분석플랫폼구축_제안서.pdf', 'total_pages': 113, 'format': 'PDF 1.7', 'title': '합치기_2', 'author': '유선철', 'subject': '', 'keywords': '', 'moddate': '2024-06-09T21:57:34+09:00', 'trapped': '', 'page': 51}, page

In [6]:
# Step 6 : 프롬프트 생성 
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean,and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)

In [7]:
# Step 7 : 언어 모델 (LLM) 생성
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")

C:\Users\morcs\AppData\Local\Temp\ipykernel_39084\2227855636.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [8]:
# 단계 8: 체인(Chain) 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "포항에서 열리는 축제의 이름이 뭐야?"
response = chain.invoke(question)
print(response)

청년 문화 페스티벌, 붐업 페스티벌, -입니다.


In [10]:
question = "청년 문화 페스티벌의 2024년 방문자수가 몇명이야?"
response = chain.invoke(question)
print(response)

당대에만 있는 청년 문화 페스티벌은 청소년을 위한 장르 페스티벌로, 2024년 10월에 개장했으며, 2일간에 5만 명 이상의 청소년이 방문했습니다.
